# **Import Tools**

In [ ]:
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from tensorflow.keras.models import load_model
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import tensorflow as tf
tf.random.set_seed(1234)
AUTO = tf.data.experimental.AUTOTUNE
!pip -q install tensorflow-datasets==1.2.0
import sys
import tensorflow_datasets as td
import csv
import numpy as np 
import pandas as pd
from datasets import load_dataset
!pip -q install datasets
import os
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
import spacy
import random
from transformers import AutoModelForCausalLM,  AutoTokenizer
import warnings
import logging
from collections import Counter
import keras
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.initializers import Constant
from keras.layers import (LSTM, Embedding, BatchNormalization, Dense, TimeDistributed, Dropout, Bidirectional,Flatten, GlobalMaxPool1D)
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, SpatialDropout1D, Conv1D, LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from spacy.util import compounding
from spacy.util import minibatch
from collections import defaultdict
from collections import Counter
from sklearn.metrics import (precision_score, recall_score, f1_score, classification_report, accuracy_score, confusion_matrix)

# **Read Data**

In [ ]:
mul_em = pd.read_csv('/kaggle/input/multiclass-emotion-model-dataset/data_train.csv')

emotion_dataset = pd.read_csv('/kaggle/input/emotion-dataset/training.csv')
tex = {0:'sadness', 1:'joy', 2:'love', 3:'anger', 4:'fear', 5:'surprise'}
emotion_dataset['emotion'] = emotion_dataset['label'].map(tex)
new_col1 = ['Emotion','Text']
c1 = emotion_dataset[['emotion','text']]
c1.columns = new_col1
emotion_dataset = c1

edos = pd.read_csv('/kaggle/input/edos-1m-data-sentiment-analysis/EDOS 1M.csv')
column1 = ['Emotion','Text']
s_c1 = edos[['eb+_emot','uttr']]
s_c1.columns = column1
edos = s_c1

isear = pd.read_csv('/kaggle/input/isear-dataset/eng_dataset.csv')
column3 = ['Emotion', 'Text']
s_c3 = isear[['sentiment', 'content']]
s_c3.columns = column3
isear = s_c3

data = pd.concat([emotion_dataset,mul_em,edos,isear], ignore_index=True)

# Fixed Max Sequence Length

In [ ]:
m_len = 60
data = data[(data['Text'].str.len() <= m_len)]
data.reset_index(drop=True, inplace=True)

# Drop Duplicates

In [ ]:
data = data.drop_duplicates(subset='Text', keep='first')
data.reset_index(drop=True, inplace=True)

# Select Emotions

In [ ]:
t_cou = {
    'joyful': 10000,
    'sad': 10000,
    'hopeful': 10000,
    'afraid': 10000,
    'caring': 10000,
    'angry': 10000,
    'proud': 10000,
    'guilty': 7613,               
    'terrified': 10000,
    'faithful': 6705,
    'impressed': 10000,
    'lonely': 9781,
    'annoyed': 10000,
    'wishing': 10000, 
    'surprised': 10000,
}
n_data = []

for emotion, count in t_cou.items():
    fil_rows = data[data['Emotion'] == emotion].head(count)
    n_data.append(fil_rows)
data = pd.concat(n_data, ignore_index=True)
data = data.sample(frac=1).reset_index(drop=True)

# **Text Cleaning**

In [ ]:
def c_corpus(x):
    x = str(x).lower()
    x = re.sub('\[.*?\]', '', x)
    x = re.sub('\n', '', x)
    x = re.sub('\w*\d\w*', '', x)
    x = re.sub('<.*?>+', '', x)
    return x
data['Text'] = data['Text'].apply(c_corpus)

# **LabelEncoder**

In [ ]:
c = { 'joyful': 0,'sad': 1,'hopeful': 2,'afraid': 3,'caring': 4,'angry': 5,'proud': 6,'guilty': 7,'terrified': 8,'faithful': 9,'impressed': 10,'lonely': 11,'annoyed': 12,'wishing': 13,'surprised': 14}

data['Emotion'] = data['Emotion'].map(c)

In [ ]:
a = LabelEncoder()
a.fit(data['Emotion'])
data['Emotion'] = a.transform(data['Emotion'])

# **Split Data**

In [ ]:
x1 = data['Text']
y1 = data['Emotion']
x_train, x_test, y_train, y_test = train_test_split(x1, y1, random_state=42)

# **TfidfVectorizer**

In [ ]:
vec = TfidfVectorizer()
vec.fit(x_train)
x_train = vec.transform(x_train)
x_test  = vec.transform(x_test)

# LinearSVC

In [ ]:
from sklearn.svm import LinearSVC
s_vc = LinearSVC()
s_model = s_vc.fit(x_train, y_train)
s_pre = s_model.predict(x_test)
s_score = round(accuracy_score(y_test, s_pre) * 100, 2)
print("The accuracy score of LinearSVC is = ",s_score)

In [ ]:
m_fi = "/kaggle/input/updated-svc/updated_svc_model.pkl"
with open(m_fi, 'rb') as m_fi:
    svc = pickle.load(m_fi)

In [ ]:
MNB = MultinomialNB()
MNB.fit(x_train, y_train)
y_p = MNB.predict(x_test)
acc = accuracy_score(y_test, y_p)
print(f"Test accuracy: {acc:.4f}")

In [ ]:
"""
p_grid = {
    'loss': ['hinge', 'log', 'modified_huber'],
    'penalty': ['l1', 'l2', 'elasticnet'],
    'alpha': [0.0001, 0.001, 0.01],
    'max_iter': [1000, 2000, 3000]
}

g_search = GridSearchCV(sgd, pa_grid, cv=5)
print("Best Hyperparameters: ", g_search.best_params_)
print("Best Score: ", g_search.best_score_)
"""
s_gd = SGDClassifier()
s_gd.fit(x_train, y_train)
pre = s_gd.predict(x_test)
acc = accuracy_score(y_test, pre)*100
print(f"Test accuracy of Stochastic Gradient Descent: {acc:.4f}")

# Pre-trained Model

In [ ]:
! pip -q install transformers

In [ ]:
from transformers import pipeline
pre_model = pipeline( model="facebook/bart-large-mnli",framework="pt",)

# LSTM

In [ ]:
#c = { 'joyful': 0,'sad': 1,'hopeful': 2,'afraid': 3,'caring': 4,'angry': 5,'proud': 6,'guilty': 7,'terrified': 8,'faithful': 9,'impressed': 10,'lonely': 11,'annoyed': 12,'wishing': 13,'surprised': 14}

#data['Emotion'] = data['Emotion'].map(c)

In [ ]:
x2 = data['Text']
y2 = data['Emotion']
x1_train, x1_test, y1_train, y1_test = train_test_split(x2, y2, random_state=42)
t = Tokenizer()
t.fit_on_texts(x1_train)

In [ ]:
l_model = load_model("/kaggle/input/updated-bilstm/updated_bilstm_model.h5")

# Emotion Detection Hybrid Approach

In [ ]:

a = { 0:'joyful',1:'sad',2:'hopeful',3:'afraid',4:'caring',5:'angry',6:'proud',7:'guilty',8:'terrified',9:'faithful',10:'impressed',11:'lonely',12:'annoyed',13:'wishing',14:'surprised'}
def pre_train(user_input): 
    r = pre_model(
    user_input,
    candidate_labels = ['joyful','sad','hopeful','afraid','caring','angry','proud','guilty','terrified','faithful','impressed','lonely','annoyed','wishing','surprised'],
    )
    em = r["scores"].index(max(r["scores"]))
    return r["labels"][em]      
def bi_lstm(user_input):
    s = [user_input]
    x1 = pad_sequences(t.texts_to_sequences(s),maxlen = 60)
    s = l_model.predict(x1, verbose=0)
    p_i = np.argmax(s)
    p_l = a[p_i] 
    return p_l

def lsv(user_input):
    s = [user_input]
    s = vec.transform(s)
    senti = s_model.predict(s)
    p_sentiment = senti[0] 
    return a[p_sentiment]

def emotion_predict_hybrid_model(tx):
    e = [pre_train(tx),lsv(tx), bi_lstm(tx)]
    e_count = Counter(e)
    emotion, count = e_count.most_common(1)[0]
    if count >= 2:
        return emotion
    else: 
        return e[0]

In [ ]:
"""
def sgd(user_input):
    s = [user_input]
    s = vec.transform(s)
    senti = s_gd.predict(s)
    p_sentiment = senti[0] 
    x = a[p_sentiment]
    #print("sgd: ",x)
    return x

def m_nb(user_input):
    s = [user_input]
    s = vec.transform(s)
    senti = MNB.predict(s)
    p_sentiment = senti[0] 
    x = a[p_sentiment]
    #print("mnb: ",x)
    return x
"""

## DialoGPT Small and Emotion prediction

In [ ]:
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")

In [ ]:
def dialogpt_small_predict(txt):
    logging.getLogger("transformers").setLevel(logging.ERROR)
    tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small')
    model = AutoModelForCausalLM.from_pretrained('/kaggle/input/model-checkpoint-31500/model_checkpoint-31500')
    input_id = tokenizer.encode(txt + tokenizer.eos_token, return_tensors='pt')
    ids = model.generate(input_id, max_length=100,pad_token_id=tokenizer.eos_token_id,no_repeat_ngram_size=3,do_sample=True,top_k = 5,top_p=0.7,temperature = 0.8)
    k = tokenizer.decode(ids[:, input_id.shape[-1]:][0],skip_special_tokens=True)
    return k


In [ ]:
while True:
    n = input("User: ")

    if n == "exit":
        break
    n = re.sub(r"['\"].*?['\"]", '', n)
    u = emotion_predict_hybrid_model(n)
    b = dialogpt_small_predict(n)
    print(f"Chatbot: {u}, {b}\n")

## DialoGPT Medium and Emotion prediction

In [ ]:
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")

In [ ]:
def dialogpt_medium_predict(txt):
    logging.getLogger("transformers").setLevel(logging.ERROR)
    tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-medium')
    model = AutoModelForCausalLM.from_pretrained('/kaggle/input/dialogpt-medium-checkpoint/DialoGPT_MEDIUM_checkpoint')
    input_id = tokenizer.encode(txt + tokenizer.eos_token, return_tensors='pt')
    ids = model.generate(input_id, max_length=100,pad_token_id=tokenizer.eos_token_id,no_repeat_ngram_size=3,do_sample=True,top_k = 5,top_p=0.7,temperature = 0.8)
    k = tokenizer.decode(ids[:, input_id.shape[-1]:][0],skip_special_tokens=True)
    return k

In [ ]:
while True:
    n = input("User: ")

    if n == "exit":
        break
    n = re.sub(r"['\"].*?['\"]", '', n)
    u = emotion_predict_hybrid_model(n)
    b = dialogpt_medium_predict(n)
    print(f"Chatbot: {u}, {b}\n")

## Transformer and Emotion prediction

In [ ]:
d = load_dataset("roskoN/dailydialog")

c = d['train']['utterances']
c = [u for sublist in c for u in sublist]
prompts = c[0::2]
responses = c[1::2]

file = "daily_dialog.csv"
with open(file, mode='w', newline='') as cs:
    w = csv.writer(cs)
    w.writerow(["Prompt", "Response"]) 

    for prompt, response in zip(prompts, responses):
        w.writerow([prompt.strip(), response.strip()])
        
df = pd.read_csv(file)
#df = df[:1000]

In [ ]:
d = load_dataset("benjaminbeilharz/empathetic_dialogues_for_lm")

c = d['train']['conv']
c = [u for sublist in c for u in sublist]
prompts = c[0::2]
responses = c[1::2]

file = "empathetic_dialogues.csv"
with open(file, mode='w', newline='') as cs:
    w = csv.writer(cs)
    w.writerow(["Prompt", "Response"]) 

    for prompt, response in zip(prompts, responses):
        w.writerow([prompt.strip(), response.strip()])
        
dt = pd.read_csv(file)
#dt = dt[:1000]

In [ ]:
dat = load_dataset("allenai/prosocial-dialog")

t_data = dat['train'].to_pandas()

column = ['context', 'response']
da = t_data[column]

f = "prosocial_dialog.csv"
da.to_csv(f, index=False)

new_column1 = ['Prompt','Response']
sel_c1 = da[['context','response']]
sel_c1.columns = new_column1
da = sel_c1
#da = da[:1000]

In [ ]:
concat_q = pd.concat([df['Prompt'], dt['Prompt'], da['Prompt']], ignore_index=True)
concat_q.dropna(inplace=True)
prompt = concat_q.tolist()

concat_a = pd.concat([df['Response'], dt['Response'], da['Response']], ignore_index=True)
concat_a.dropna(inplace=True)
response = concat_a.tolist()

print(len(prompt))
print(len(response))

In [ ]:
max_len = 60
max_sample = 220000
bat_size = 64
buf_size = 170000
layer = 2
model_di = 512
head = 8
unit = 128
drop = 0.1

In [ ]:
def t_pre(s):
    s = s.lower().strip()
    s= re.sub(r"([?.!,])", r" \1 ", s)
    s = re.sub(r"[^a-zA-Z?.!,]+", " ", s)
    s = re.sub(r'[" "]+', " ", s)
    s = s.strip()
    return s

prompt = [t_pre(s) for s in prompt]
response = [t_pre(s) for s in response]

In [ ]:
tokenizer = td.features.text.SubwordTextEncoder.build_from_corpus(prompt + response, target_vocab_size=8000)

s_token, e_token = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

v_size = tokenizer.vocab_size + 2

t_prompt, t_response = [], []

for (i, j) in zip(prompt, response):
    i = s_token + tokenizer.encode(i) + e_token
    j = s_token + tokenizer.encode(j) + e_token
    if len(i) <= max_len and len(j) <= max_len:
        t_prompt.append(i)
        t_response.append(j)

prompt = tf.keras.preprocessing.sequence.pad_sequences(t_prompt, maxlen=max_len, padding='post')
response = tf.keras.preprocessing.sequence.pad_sequences(t_response, maxlen=max_len, padding='post')

In [ ]:
d = tf.data.Dataset.from_tensor_slices(({ 'inputs': prompt,'dec_inputs': response[:, :-1] },{'outputs': response[:, 1:]},))
d = d.cache()
d = d.shuffle(buf_size)
d = d.batch(bat_size)
d = d.prefetch(tf.data.experimental.AUTOTUNE)
d_size = len(d)
t_size = int(0.8 * d_size)
train_data = d.take(t_size)
val_data = d.skip(t_size)

In [ ]:
class MultiHeadAttentionLayer(tf.keras.layers.Layer):

    def __init__(self, mod_d, head, **kwargs):
        super(MultiHeadAttentionLayer, self).__init__(**kwargs)
        self.head = head
        self.mod_d = mod_d
        assert mod_d % self.head == 0
        self.depth = mod_d // self.head

        self.q_den = tf.keras.layers.Dense(units=mod_d)
        self.k_den = tf.keras.layers.Dense(units=mod_d)
        self.v_den = tf.keras.layers.Dense(units=mod_d)
        self.dense = tf.keras.layers.Dense(units=mod_d)

    def get_config(self):
        c = super(MultiHeadAttentionLayer,self).get_config()
        c.update({ "head":self.head,"mod_d":self.mod_d,})
        return c

    def splitheads(self, inputs, b_size):
        inputs = tf.keras.layers.Lambda(lambda inputs:tf.reshape(inputs, shape=(b_size, -1, self.head, self.depth)))(inputs)
        r = tf.keras.layers.Lambda(lambda inputs: tf.transpose(inputs, perm=[0, 2, 1, 3]))(inputs)
        return r

    def call(self, inputs):
        q, k, v, m = inputs['query'], inputs['key'], inputs['value'], inputs['mask']
        b_size = tf.shape(q)[0]
        
        #linear layer
        q = self.q_den(q)
        k = self.k_den(k)
        v = self.v_den(v)
        
        #split head
        q = self.splitheads(q, b_size)
        k = self.splitheads(k, b_size)
        v = self.splitheads(v, b_size)
         
        #scaled_dot_product attention
        q_k = tf.matmul(q, k, transpose_b=True)
        
        depth = tf.cast(tf.shape(k)[-1], tf.float32)
        
        logit = q_k / tf.math.sqrt(depth)
        
        if m is not None:
            logit += (m * -1e9)
        a_wei = tf.nn.softmax(logit, axis=-1)
        s_att = tf.matmul(a_wei, v)
        s_att = tf.keras.layers.Lambda(lambda s_att: tf.transpose(s_att, perm=[0, 2, 1, 3]))(s_att)
        
         #concatenation
        c_att = tf.keras.layers.Lambda(lambda s_att: tf.reshape(s_att,(b_size, -1, self.mod_d)))(s_att)
        
        # final layer
        a = self.dense(c_att)
        return a

In [ ]:
class PositionalEncoding(tf.keras.layers.Layer):

    def __init__(self, p, mod_d, **kwargs):
        super(PositionalEncoding, self).__init__(**kwargs)
        self.p = p
        self.mod_d = mod_d
        self.en_pos = self.en_positional(p, mod_d)

    def get_config(self):
        c = super(PositionalEncoding, self).get_config()
        
        c.update({"p": self.p,"mod_d": self.mod_d,})
        
        return c

    def getangle(self, p, j, mod_d):
        a =  tf.pow(10000, (2 * (j // 2)) / tf.cast(mod_d, tf.float32))
        ang = 1 / a
        return p * ang

    def en_positional(self, p, mod_d):
        r_ang = self.getangle(p = tf.range(p, dtype=tf.float32)[:, tf.newaxis],j = tf.range(mod_d, dtype = tf.float32)[tf.newaxis, :],mod_d = mod_d)
        sin = tf.math.sin(r_ang[:, 0::2])
        cos = tf.math.cos(r_ang[:, 1::2])
        en_pos = tf.concat([sin, cos], axis=-1)
        en_pos = en_pos[tf.newaxis, ...]
        t = tf.cast(en_pos, tf.float32)
        return t

    def call(self, j):
        r = j + self.en_pos[:, :tf.shape(j)[1], :]
        return r


In [ ]:
def EncoderLayer(u, mod_d, head, drop, name="encoder_layer"):
    inp = tf.keras.Input(shape=(None, mod_d), name="inputs")
    p_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    a = MultiHeadAttentionLayer(mod_d, head, name="attention")({'query': inp,'key': inp,'value': inp,'mask': p_mask})
    a = tf.keras.layers.Dropout(rate=drop)(a)
    a_at = tf.keras.layers.add([inp,a])
    a = tf.keras.layers.LayerNormalization(epsilon=1e-6)(a_at)

    op = tf.keras.layers.Dense(units=u, activation='relu')(a)
    op = tf.keras.layers.Dense(units=mod_d)(op)
    op = tf.keras.layers.Dropout(rate=drop)(op)
    a_at = tf.keras.layers.add([a,op])
    op = tf.keras.layers.LayerNormalization(epsilon=1e-6)(a_at)

    t = tf.keras.Model(inputs=[inp, p_mask], outputs = op, name = name)
    return t



def Encoder(v_size,lay, u,mod_d,head,drop, name="encoder"):
    inp = tf.keras.Input(shape=(None,), name="inputs")
    p_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    em = tf.keras.layers.Embedding(v_size, mod_d)(inp)
    em *= tf.keras.layers.Lambda(lambda mod_d: tf.math.sqrt(tf.cast(mod_d, tf.float32)))(mod_d)
    em = PositionalEncoding(v_size,mod_d)(em)

    op = tf.keras.layers.Dropout(rate=drop)(em)

    for i in range(lay):
        op = EncoderLayer(u,mod_d,head,drop,name="encoder_layer_{}".format(i),)([op, p_mask])

    k = tf.keras.Model(inputs=[inp, p_mask], outputs=op, name="encoder")
    return k

In [ ]:
def DecoderLayer(u, mod_d, head, drop, name="decoder_layer"):
    inp = tf.keras.Input(shape=(None, mod_d), name="inputs")
    e_op = tf.keras.Input(shape=(None, mod_d), name="encoder_outputs")
    ahead_mask = tf.keras.Input(shape=(1, None, None), name="look_ahead_mask")
    p_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    at1 = MultiHeadAttentionLayer(mod_d, head, name="attention_1")(inputs={'query': inp,'key': inp,'value': inp,'mask': ahead_mask})
    add_at = tf.keras.layers.add([at1,inp])
    at1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(add_at)

    at2 = MultiHeadAttentionLayer(mod_d, head, name="attention_2")(inputs={'query': at1,'key': e_op,'value': e_op,'mask': p_mask})
    at2 = tf.keras.layers.Dropout(rate=drop)(at2)
    add_at = tf.keras.layers.add([at2,at1])
    at2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(add_at)

    op = tf.keras.layers.Dense(units=u, activation='relu')(at2)
    op = tf.keras.layers.Dense(units=mod_d)(op)
    op = tf.keras.layers.Dropout(rate=drop)(op)
    add_at = tf.keras.layers.add([op,at2])
    op = tf.keras.layers.LayerNormalization(epsilon=1e-6)(add_at)

    t = tf.keras.Model(inputs=[inp, e_op, ahead_mask, p_mask],outputs = op,name = name)
    
    return t

def Decoder(v, lay, u,mod_d,head,drop,name='decoder'):
    inp = tf.keras.Input(shape=(None,), name='inputs')
    e_op = tf.keras.Input(shape=(None, mod_d), name='encoder_outputs')
    ahead_mask = tf.keras.Input(shape=(1, None, None), name='look_ahead_mask')
    p_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    em = tf.keras.layers.Embedding(v, mod_d)(inp)
    em *= tf.keras.layers.Lambda(lambda mod_d: tf.math.sqrt(tf.cast(mod_d, tf.float32)))(mod_d)
    em = PositionalEncoding(v, mod_d)(em)

    ot = tf.keras.layers.Dropout(rate=drop)(em)

    for j in range(lay):
        ot = DecoderLayer(u,mod_d,head,drop,name='decoder_layer_{}'.format(j),)(inputs=[ot, e_op, ahead_mask, p_mask])

    p = tf.keras.Model(inputs=[inp, e_op, ahead_mask, p_mask],outputs = ot,name = 'decoder')
    return p

In [ ]:
def padding_m(i):
    m = tf.cast(tf.math.equal(i, 0), tf.float32)
    t = m[:, tf.newaxis, tf.newaxis, :]
    return t

def look_ahead_m(i):
    s_len = tf.shape(i)[1]
    ahead_m = 1 - tf.linalg.band_part(tf.ones((s_len, s_len)), -1, 0)
    p = padding_m(i)
    t = tf.maximum(ahead_m, p)
    return t

In [ ]:
inp = tf.keras.Input(shape=(None,), name="inputs")

d_input = tf.keras.Input(shape=(None,), name="dec_inputs")
    
pad_mask_en = tf.keras.layers.Lambda(padding_m, output_shape=(1, 1, None), name="enc_padding_mask")(inp)

ahead_mask = tf.keras.layers.Lambda(look_ahead_m, output_shape=(1, None, None), name="look_ahead_mask")(d_input)

pad_mask_de = tf.keras.layers.Lambda(padding_m, output_shape=(1, 1, None), name="dec_padding_mask")(inp)

out_en = Encoder(v_size,layer,unit,model_di,head,drop,)(inputs=[inp, pad_mask_en])

out_de = Decoder(v_size,layer,unit,model_di,head,drop,)(inputs=[d_input, out_en, ahead_mask, pad_mask_de])

op = tf.keras.layers.Dense(units=v_size, name="outputs")(out_de)

model = tf.keras.Model(inputs=[inp, d_input], outputs=op, name="transformer")

In [ ]:
f = '/kaggle/input/transformer-model/transformer_model.h5'
transformer_model = tf.keras.models.load_model(
    f,
    custom_objects={
        "MultiHeadAttentionLayer": MultiHeadAttentionLayer,
        "PositionalEncoding": PositionalEncoding,
    },
    compile=False,
)

In [ ]:
def transformer_response_generation(s):
    s = t_pre(s)
    s = tf.expand_dims(s_token + tokenizer.encode(s) + e_token, axis=0)
    out = tf.expand_dims(s_token, 0)
    for i in range(max_len):
        pre = transformer_model(inputs=[s, out], training=False)
        pre = pre[:, -1:, :]
        pre_id = tf.cast(tf.argmax(pre, axis=-1), tf.int32)
        if tf.equal(pre_id, e_token[0]):
            break
        out = tf.concat([out, pre_id], axis=-1)
    
    p = tf.squeeze(out, axis=0)
    pre_prompt = tokenizer.decode([i for i in p if i < tokenizer.vocab_size])
    return pre_prompt
  

In [ ]:
while True:
    n = input("User: ")
    if n == "exit":
        break
    n = re.sub(r"['\"].*?['\"]", '', n)
    u = emotion_predict_hybrid_model(n)
    b = transformer_response_generation(n)
    print(f"Chatbot: {u}, {b}\n")

In [ ]:
store_conversation = []
v = ""
store_conversation.append(v)
flag = 0
while True:
    n = input("User: ")
    if n == "exit":
        break
    n = re.sub(r"['\"].*?['\"]", '', n)
    u = emotion_predict_hybrid_model(n)
    #if flag == 0:
        #flag = 1
        #text1 = n
    #else:
        #store_conversation.append(n)
        #text1 = n + " " + store_conversation[-1]
        #print("Text : ", text1)
        #text1 = store_conversation[-1]+ " " +n
        #store_conversation.append(n)
        #print("Text : ", text1)
    b = transformer_response_generation(n)
    print(f"Chatbot: {u}, {b}\n")